****Phase-1****

In [1]:
%pip install pandas
%pip install pyarrow
%pip install fsspec
%pip install huggingface_hub
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_parquet("hf://datasets/CalebE/new_mental_health_conversations_all1/data/train-00000-of-00001.parquet")

In [3]:
df_small = df.head(50)  # Change to 50 if needed

# Save the reduced dataset (optional)
df_small.to_parquet("small_dataset.parquet", index=False)

# Display the reduced dataset
# print(df_small)
# print(df.head())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13358 entries, 0 to 13357
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  13358 non-null  object
 1   output       13358 non-null  object
 2   input        13358 non-null  object
dtypes: object(3)
memory usage: 313.2+ KB
None


In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
        tokens = word_tokenize(text)  # Tokenize
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        return " ".join(tokens)  # Convert back to string
    return text

df["clean_instruction"] = df["instruction"].apply(clean_text)
df["clean_input"] = df["input"].apply(clean_text)
df["clean_output"] = df["output"].apply(clean_text)

print(df[["clean_instruction", "clean_input", "clean_output"]].head())


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
                                   clean_instruction  \
0         im feeling really anxious lately dont know   
1                         think partner may cheating   
2  im feeling really overwhelmed work school dont...   
3  im trouble sleeping im constantly tired think ...   
4        ive feeling really anxious lately dont know   

                                         clean_input  \
0  licensed psychologist please provide patient h...   
1  licensed psychologist please provide patient h...   
2  licensed psychologist please provide patient h...   
3  licensed psychologist please provide patient h...   
4  licensed psychologist please provide patient h...   

                                        clean_output  
0  common feel anxious times many reaso

In [5]:
from nltk.tokenize import sent_tokenize

# Apply tokenization
df["tokenized_instruction"] = df["clean_instruction"].apply(word_tokenize)
df["tokenized_sentences"] = df["clean_instruction"].apply(sent_tokenize)

print(df[["tokenized_instruction", "tokenized_sentences"]].head())


                               tokenized_instruction  \
0  [im, feeling, really, anxious, lately, dont, k...   
1                    [think, partner, may, cheating]   
2  [im, feeling, really, overwhelmed, work, schoo...   
3  [im, trouble, sleeping, im, constantly, tired,...   
4  [ive, feeling, really, anxious, lately, dont, ...   

                                 tokenized_sentences  
0       [im feeling really anxious lately dont know]  
1                       [think partner may cheating]  
2  [im feeling really overwhelmed work school don...  
3  [im trouble sleeping im constantly tired think...  
4      [ive feeling really anxious lately dont know]  


In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")

def pos_tagging(text):
    doc = nlp(text)
    return [(token.text, token.pos_) for token in doc]

df["pos_tags"] = df["clean_instruction"].apply(pos_tagging)

print(df[["clean_instruction", "pos_tags"]].head())


                                   clean_instruction  \
0         im feeling really anxious lately dont know   
1                         think partner may cheating   
2  im feeling really overwhelmed work school dont...   
3  im trouble sleeping im constantly tired think ...   
4        ive feeling really anxious lately dont know   

                                            pos_tags  
0  [(i, PRON), (m, AUX), (feeling, VERB), (really...  
1  [(think, VERB), (partner, NOUN), (may, AUX), (...  
2  [(i, PRON), (m, AUX), (feeling, VERB), (really...  
3  [(i, PRON), (m, VERB), (trouble, NOUN), (sleep...  
4  [(i, PRON), (ve, AUX), (feeling, VERB), (reall...  


In [7]:
from transformers import BertTokenizer, BertModel
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased").to(device)  # Move model to GPU

def get_bert_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    tokens = {key: value.to(device) for key, value in tokens.items()}  # Move input tensors to GPU
    
    with torch.no_grad():
        output = model(**tokens)
    
    return output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Move output back to CPU for Pandas

# Apply function to DataFrame
df["bert_embedding"] = df["clean_instruction"].apply(get_bert_embedding)

print(df["bert_embedding"].head())


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

0    [0.04772697, 0.06270748, 0.39699778, 0.0469553...
1    [0.06670084, -0.06419288, 0.087617114, -0.0953...
2    [0.15714815, 0.38649198, 0.30842364, -0.025939...
3    [-0.018088518, 0.32670245, 0.51900166, -0.2724...
4    [0.06580624, 0.11805924, 0.31043625, 0.1069824...
Name: bert_embedding, dtype: object


In [10]:
df.to_parquet("processed_mental_health_data.parquet", index=False)


***Phase-2***

In [1]:
%pip install langchain
%pip install langchain_community
%pip install chromadb
%pip install llama-cpp-python
%pip install -U bitsandbytes
%pip install faiss-gpu

  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 31.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    F

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from transformers import pipeline

In [7]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
# Download necessary NLTK dat
nltk.download("punkt")
nltk.download("stopwords")

df = pd.read_parquet("hf://datasets/CalebE/new_mental_health_conversations_all1/data/train-00000-of-00001.parquet")

# ✅ Merge `input` and `output` to create meaningful text for retrieval
df["combined_text"] = df["instruction"].fillna("") + " " + df["input"].fillna("") + " " + df["output"].fillna("")

# ✅ Remove empty texts
df = df.dropna(subset=["combined_text"])

# ✅ Convert into LangChain Document format
documents = [Document(page_content=text) for text in df["combined_text"].tolist()]

# ✅ Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)


# Define stopwords
stop_words = set(stopwords.words("english"))

# Define text-cleaning function
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
        tokens = word_tokenize(text)  # Tokenize
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        return " ".join(tokens)  # Convert back to string
    return text

# Apply cleaning
df["clean_input"] = df["input"].apply(clean_text)
df["clean_output"] = df["output"].apply(clean_text)
df["clean_instruction"] = df["instruction"].apply(clean_text)

# Check processed data
print(df[["clean_instruction", "clean_input", "clean_output"]].head())


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                                   clean_instruction  \
0         im feeling really anxious lately dont know   
1                         think partner may cheating   
2  im feeling really overwhelmed work school dont...   
3  im trouble sleeping im constantly tired think ...   
4        ive feeling really anxious lately dont know   

                                         clean_input  \
0  licensed psychologist please provide patient h...   
1  licensed psychologist please provide patient h...   
2  licensed psychologist please provide patient h...   
3  licensed psychologist please provide patient h...   
4  licensed psychologist please provide patient h...   

                                        clean_output  
0  common feel anxious times ma

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb
import os

# Define storage path
DB_DIR = os.path.join(os.getcwd(), "mental_health_db")

# Load embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert text into document format
from langchain.docstore.document import Document

documents = [Document(page_content=text) for text in df["clean_instruction"]]

# Store in ChromaDB
vectorstore = Chroma.from_documents(
    documents,
    embedding_model,
    persist_directory=DB_DIR
)

# Persist the database
vectorstore.persist()


<ipython-input-8-b1f87fd0dd91>:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [9]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

query = "How can I manage anxiety?"
retrieved_docs = retriever.get_relevant_documents(query)

# Display retrieved docs
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:\n{doc.page_content}\n")


Document 1:
im really struggling anxiety manage

Document 2:
im really struggling anxiety interfering daily life manage

Document 3:
im struggling anxiety social situations avoiding altogether manage anxiety

Document 4:
im trouble anxiety dont want take medication manage anxiety without medication

Document 5:
feel like cant control anxiety



<ipython-input-9-dc03a6aa411a>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


** Chain-of-Thought (CoT) Prompting**

In [10]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",  # Offload intelligently (some layers to CPU if needed)
    torch_dtype="auto",  # Use automatic precision
    max_new_tokens=512,  # Reduce max tokens
    temperature=0.75,
    top_p=1
)


NameError: name 'model' is not defined

In [14]:
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-14-b97a330b5f24>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# ✅ Load dataset (Parquet file)
df = pd.read_parquet("hf://datasets/CalebE/new_mental_health_conversations_all1/data/train-00000-of-00001.parquet")

# ✅ Merge `input` and `output` to create meaningful text for retrieval
df["combined_text"] = df["instruction"].fillna("") + " " + df["input"].fillna("") + " " + df["output"].fillna("")

# ✅ Remove empty texts
df = df.dropna(subset=["combined_text"])

# ✅ Convert into LangChain Document format
documents = [Document(page_content=text) for text in df["combined_text"].tolist()]

# ✅ Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# ✅ Create embeddings & store in FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding_model)
retriever = vectorstore.as_retriever()


In [13]:
import torch

In [14]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use 16-bit precision
    device_map="auto"  # Automatically offload to CPU/GPU
)


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [16]:
# ✅ Load tokenizer & model
# model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# ✅ Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Reduce for memory efficiency
    temperature=0.75,
    top_p=1,
    device_map="auto"  # Enable GPU/CPU offloading
)

# ✅ Wrap the pipeline in LangChain's HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=pipe)




tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-16-e155b5058dad>:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [19]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# ✅ Define Model Name Once
model_name = "HuggingFaceH4/zephyr-7b-beta"

# ✅ Load tokenizer & model (Only Once)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use 16-bit precision
    device_map="auto"  # Automatically offload to CPU/GPU
)

# ✅ Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Reduce for memory efficiency
    temperature=0.75,
    top_p=1,
    device_map="auto"  # Enable GPU/CPU offloading
)

# ✅ Wrap the pipeline in LangChain's HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=pipe)

# ✅ Load dataset (Parquet file)
df = pd.read_parquet("hf://datasets/CalebE/new_mental_health_conversations_all1/data/train-00000-of-00001.parquet")

# ✅ Merge `input` and `output` to create meaningful text for retrieval
df["combined_text"] = df["instruction"].fillna("") + " " + df["input"].fillna("") + " " + df["output"].fillna("")

# ✅ Remove empty texts
df = df.dropna(subset=["combined_text"])

# ✅ Convert into LangChain Document format
documents = [Document(page_content=text) for text in df["combined_text"].tolist()]

# ✅ Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# ✅ Create embeddings & store in FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding_model)
retriever = vectorstore.as_retriever()


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [22]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# ✅ Define Chain of Thought Prompt Template
cot_prompt_template = """You are an AI assistant that provides detailed and step-by-step responses based on retrieved information. 
Use logical reasoning to analyze the given context and answer the question in a structured manner.

Context:
{context}

Follow these steps:
1. Identify key details from the context.
2. Break down the reasoning step by step.
3. Provide a detailed, clear, and informative answer.

Question: {question}
Final Answer:"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template, 
    input_variables=["context", "question"]
)

# ✅ Create Retrieval QA Chain with CoT Prompting
retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Stuffing all retrieved docs in the prompt
    retriever=retriever,
    chain_type_kwargs={"prompt": cot_prompt}
)

# ✅ Query with structured input
query = "How does stress impact mental health?"
response = retrieval_chain.invoke({"query": query})  

# ✅ Beautify the output
def beautify_output(response):
    print("\n=== Final Answer ===\n")
    print(response["result"])
    # print("\n=== Source Documents ===")
    # for i, doc in enumerate(response["source_documents"], 1):
    #     print(f"\n📌 Source {i}: {doc.page_content}")

# ✅ Display the formatted response
beautify_output(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



=== Final Answer ===

You are an AI assistant that provides detailed and step-by-step responses based on retrieved information. 
Use logical reasoning to analyze the given context and answer the question in a structured manner.

Context:
engaging in stress-reducing activities such as exercise or meditation can help improve your overall mental health.

issues that may be contributing to your stress levels.

be helpful to explore any underlying issues that may be contributing to your stress.

It's important to identify the source of your stress and develop a plan to manage it.

Follow these steps:
1. Identify key details from the context.
2. Break down the reasoning step by step.
3. Provide a detailed, clear, and informative answer.

Question: How does stress impact mental health?
Final Answer: Stress, when experienced in short bursts, can be beneficial as it helps us respond to challenging situations. However, chronic stress can have negative effects on our mental health. Prolonged str

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# ✅ Define Chain of Thought (CoT) Prompt
cot_prompt_template = """You are an AI assistant that provides detailed, structured, and step-by-step responses based on retrieved information.
Use logical reasoning to analyze the given context and answer the question effectively.

Context:
{context}

**Step-by-Step Breakdown:**
1. Identify key details from the context.
2. Break down the reasoning step by step.
3. Provide a detailed, clear, and informative answer.

**Question:** {question}
**Final Answer:**"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template, 
    input_variables=["context", "question"]
)

# ✅ Define Tree of Thoughts (ToT) Prompt
tot_prompt_template = """You are an AI assistant that considers multiple reasoning paths before answering.
Use a Tree of Thoughts approach to explore different possible explanations.

Context:
{context}

**Reasoning Paths (Tree of Thoughts):**
1. **Branch 1:** Consider one possible explanation and analyze it.
2. **Branch 2:** Consider an alternative viewpoint and explore it.
3. **Branch 3:** If applicable, weigh both perspectives and decide the best response.

**Question:** {question}
**Final Answer (Based on Best Reasoning Path):**"""

tot_prompt = PromptTemplate(
    template=tot_prompt_template, 
    input_variables=["context", "question"]
)

# ✅ Define Graph of Thoughts (GoT) Prompt
got_prompt_template = """You are an AI assistant that connects multiple reasoning paths into a structured answer.
Use a Graph of Thoughts approach to analyze different perspectives and combine them for the best response.

Context:
{context}

**Graph of Thoughts Analysis:**
1. Identify the key ideas from the retrieved documents.
2. Find connections between different concepts.
3. Integrate multiple insights into a cohesive and well-structured response.

**Question:** {question}
**Final Answer (Combining Multiple Insights):**"""

got_prompt = PromptTemplate(
    template=got_prompt_template, 
    input_variables=["context", "question"]
)

# ✅ Create Retrieval Chains for CoT, ToT, and GoT
retrieval_chain_cot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  
    retriever=retriever,
    chain_type_kwargs={"prompt": cot_prompt}
)

retrieval_chain_tot = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": tot_prompt}
)

retrieval_chain_got = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": got_prompt}
)

# ✅ Query and Get Responses
query = "How does stress impact mental health?"

response_cot = retrieval_chain_cot.invoke({"query": query})
response_tot = retrieval_chain_tot.invoke({"query": query})
response_got = retrieval_chain_got.invoke({"query": query})

# ✅ Beautify the output
def beautify_output(response, method_name):
    print(f"\n=== {method_name} Final Answer ===\n")
    print(response["result"])
    print("\n=== Source Documents ===")
    for i, doc in enumerate(response["source_documents"], 1):
        print(f"\n📌 Source {i}: {doc.page_content}")

# ✅ Display the responses
beautify_output(response_cot, "Chain of Thought (CoT)")
beautify_output(response_tot, "Tree of Thoughts (ToT)")
beautify_output(response_got, "Graph of Thoughts (GoT)")
